In [1]:
import warnings
warnings.filterwarnings('ignore')


In [2]:
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from pymongo import MongoClient
import os
import sys

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [3]:
# load the config from the .env file
load_dotenv()


True

In [4]:
MONGODB_URI = os.environ['MONGODB_URI']

In [5]:
# Connect to the database engine
client = MongoClient(MONGODB_URI)

In [6]:
# connect to the project db
db = client['ExpectLife']

In [7]:
# get a reference to the data collection
data = db['supervised_ml']

In [8]:
# Create a dataframe from the collection
df = pd.DataFrame(list(data.find()))
df

,_id,,Life Expectancy,Alcohol Consumption,Tobacco Consumption,GDP per Capita,Goverment Expenditure per Capita,Private Expenditure per Capita,sss_depth
0,63bcb54584ec12bf12dfbee4,Albania 2000,75,6.57,35.0,1960.881946,105.4568302,166.5878859,2
1,63bcb54584ec12bf12dfbee5,Albania 2001,75,6.786,34.26,2143.526278,115.7979181,173.4881614,2
2,63bcb54584ec12bf12dfbee6,Albania 2002,75,7.002,33.52,2247.497491,117.7658883,187.6811743,2
3,63bcb54584ec12bf12dfbee7,Albania 2003,75,7.218,32.78,2380.643997,130.028157,185.7811959,2
4,63bcb54584ec12bf12dfbee8,Albania 2004,80,7.434,32.04,2522.448714,149.1384562,188.5876134,2
...,...,...,...,...,...,...,...,...,...
3375,63bcb54884ec12bf12dfcc13,Zimbabwe 2015,60,4.92,13.6,1410.329174,41.56881467,109.6745408,1
3376,63bcb54884ec12bf12dfcc14,Zimbabwe 2016,65,4.92,13.1,1393.717011,50.2059927,105.0808457,1
3377,63bcb54884ec12bf12dfcc15,Zimbabwe 2017,65,4.67,12.6,1421.24077,54.33303618,102.4217899,1
3378,63bcb54884ec12bf12dfcc16,Zimbabwe 2018,65,4.67,12.1,1462.59028,86.31001407,115.1346444,1


In [ ]:
# Drop the database id data and refresh the indextreset_index
df = df.drop(['_id'], axis=1)
df = df.reset_index(drop=True)

In [9]:
columns = ["life_expectancy", "alcohol_consumption", "tobacco_usage",  "gdp", "govt_expenditure", "prv_expenditure", "survival_rate_male", "survival_rate_female", "sss_depth"
]

target = ["Life Expectancy"]
_

,_id,,Life Expectancy,Alcohol Consumption,Tobacco Consumption,GDP per Capita,Goverment Expenditure per Capita,Private Expenditure per Capita,sss_depth
0,63bcb54584ec12bf12dfbee4,Albania 2000,75,6.57,35.0,1960.881946,105.4568302,166.5878859,2
1,63bcb54584ec12bf12dfbee5,Albania 2001,75,6.786,34.26,2143.526278,115.7979181,173.4881614,2
2,63bcb54584ec12bf12dfbee6,Albania 2002,75,7.002,33.52,2247.497491,117.7658883,187.6811743,2
3,63bcb54584ec12bf12dfbee7,Albania 2003,75,7.218,32.78,2380.643997,130.028157,185.7811959,2
4,63bcb54584ec12bf12dfbee8,Albania 2004,80,7.434,32.04,2522.448714,149.1384562,188.5876134,2
...,...,...,...,...,...,...,...,...,...
3375,63bcb54884ec12bf12dfcc13,Zimbabwe 2015,60,4.92,13.6,1410.329174,41.56881467,109.6745408,1
3376,63bcb54884ec12bf12dfcc14,Zimbabwe 2016,65,4.92,13.1,1393.717011,50.2059927,105.0808457,1
3377,63bcb54884ec12bf12dfcc15,Zimbabwe 2017,65,4.67,12.6,1421.24077,54.33303618,102.4217899,1
3378,63bcb54884ec12bf12dfcc16,Zimbabwe 2018,65,4.67,12.1,1462.59028,86.31001407,115.1346444,1


In [10]:
df = pd.read_csv("../Clean_Data/Supervised_Machine_Learning_Data/Supervised_ML.csv", index_col = 0)
df

,Life Expectancy,Alcohol Consumption,Tobacco Consumption,GDP per Capita,Goverment Expenditure per Capita,Private Expenditure per Capita,sss_depth
Albania 2000,75,6.570,35.00,1960.881946,105.456830,166.587886,2
Albania 2001,75,6.786,34.26,2143.526278,115.797918,173.488161,2
Albania 2002,75,7.002,33.52,2247.497491,117.765888,187.681174,2
Albania 2003,75,7.218,32.78,2380.643997,130.028157,185.781196,2
Albania 2004,80,7.434,32.04,2522.448714,149.138456,188.587613,2
...,...,...,...,...,...,...,...
Zimbabwe 2015,60,4.920,13.60,1410.329174,41.568815,109.674541,1
Zimbabwe 2016,65,4.920,13.10,1393.717011,50.205993,105.080846,1
Zimbabwe 2017,65,4.670,12.60,1421.24077,54.333036,102.421790,1
Zimbabwe 2018,65,4.670,12.10,1462.59028,86.310014,115.134644,1


In [11]:
df.columns

Index(['Life Expectancy', 'Alcohol Consumption', 'Tobacco Consumption',
       'GDP per Capita', 'Goverment Expenditure per Capita',
       'Private Expenditure per Capita', 'sss_depth'],
      dtype='object')

In [12]:
# Create our features
X = pd.get_dummies(df)


# Create our target
y=df["Life Expectancy"]
X.columns

Index(['Life Expectancy', 'Alcohol Consumption', 'Tobacco Consumption',
       'Goverment Expenditure per Capita', 'Private Expenditure per Capita',
       'sss_depth', 'GDP per Capita_..', 'GDP per Capita_10013.70319',
       'GDP per Capita_10028.35435', 'GDP per Capita_100292.5685',
       ...
       'GDP per Capita_995.3416807', 'GDP per Capita_9952.916832',
       'GDP per Capita_9958.460938', 'GDP per Capita_9960.139039',
       'GDP per Capita_9965.635971', 'GDP per Capita_9966.047881',
       'GDP per Capita_9968.987254', 'GDP per Capita_9970.441715',
       'GDP per Capita_9984.146756', 'GDP per Capita_9997.686914'],
      dtype='object', length=3371)

In [13]:
X.describe()

,Life Expectancy,Alcohol Consumption,Tobacco Consumption,Goverment Expenditure per Capita,Private Expenditure per Capita,sss_depth,GDP per Capita_..,GDP per Capita_10013.70319,GDP per Capita_10028.35435,GDP per Capita_100292.5685,...,GDP per Capita_995.3416807,GDP per Capita_9952.916832,GDP per Capita_9958.460938,GDP per Capita_9960.139039,GDP per Capita_9965.635971,GDP per Capita_9966.047881,GDP per Capita_9968.987254,GDP per Capita_9970.441715,GDP per Capita_9984.146756,GDP per Capita_9997.686914
count,3380.000000,3380.000000,3380.000000,3380.000000,3380.000000,3380.000000,3380.000000,3380.000000,3380.000000,3380.000000,...,3380.000000,3380.000000,3380.000000,3380.000000,3380.000000,3380.000000,3380.000000,3380.000000,3380.000000,3380.000000
mean,72.181953,6.246839,24.120034,705.883609,372.640152,1.911243,0.000296,0.000296,0.000296,0.000296,...,0.000296,0.000296,0.000296,0.000296,0.000296,0.000296,0.000296,0.000296,0.000296,0.000296
std,9.402592,4.244134,10.305526,1037.028429,528.312417,0.848614,0.017201,0.017201,0.017201,0.017201,...,0.017201,0.017201,0.017201,0.017201,0.017201,0.017201,0.017201,0.017201,0.017201,0.017201
min,45.000000,0.003000,3.600000,0.277396,3.010181,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,65.000000,2.461500,16.380000,52.304672,73.729646,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,75.000000,5.810000,24.116667,260.518168,210.303949,2.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,80.000000,9.550000,30.185000,877.311330,503.628383,2.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,85.000000,19.950000,68.500000,6193.965917,5810.163956,5.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
# Check the balance of our target values
y.value_counts()

75    950
80    694
70    450
85    374
65    329
60    251
55    211
50     84
45     37
Name: Life Expectancy, dtype: int64

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [16]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brfc = BalancedRandomForestClassifier(random_state=1)
brfc.fit(X_train, y_train)


BalancedRandomForestClassifier(random_state=1)

In [17]:
# Calculated the balanced accuracy score
y_pred = brfc.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9241955327193394

In [18]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  9,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,  21,   0,   0,   0,   0,   0,   0,   0],
       [  0,   6,  47,   0,   0,   0,   0,   0,   0],
       [  0,   0,   2,  58,   3,   0,   0,   0,   0],
       [  0,   1,   0,   8,  62,   5,   6,   0,   0],
       [  0,   0,   0,   2,   3, 101,   6,   0,   0],
       [  0,   0,   0,   0,   5,  19, 210,   3,   0],
       [  0,   0,   0,   0,   0,   0,   4, 170,   0],
       [  0,   0,   0,   0,   0,   0,   0,   1,  93]], dtype=int64)

In [19]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

         45       1.00      1.00      1.00      1.00      1.00      1.00         9
         50       0.75      1.00      0.99      0.86      1.00      0.99        21
         55       0.96      0.89      1.00      0.92      0.94      0.87        53
         60       0.85      0.92      0.99      0.89      0.95      0.90        63
         65       0.85      0.76      0.99      0.80      0.86      0.73        82
         70       0.81      0.90      0.97      0.85      0.93      0.87       112
         75       0.93      0.89      0.97      0.91      0.93      0.86       237
         80       0.98      0.98      0.99      0.98      0.99      0.97       174
         85       1.00      0.99      1.00      0.99      0.99      0.99        94

avg / total       0.92      0.91      0.98      0.91      0.95      0.89       845



In [20]:
# List the features sorted in descending order by feature importance
# X.head()
feature_list = X.columns.array
feature_importance = brfc.feature_importances_
feature_list

<PandasArray>
[                 'Life Expectancy',              'Alcohol Consumption',
              'Tobacco Consumption', 'Goverment Expenditure per Capita',
   'Private Expenditure per Capita',                        'sss_depth',
                'GDP per Capita_..',       'GDP per Capita_10013.70319',
       'GDP per Capita_10028.35435',       'GDP per Capita_100292.5685',
 ...
       'GDP per Capita_995.3416807',       'GDP per Capita_9952.916832',
       'GDP per Capita_9958.460938',       'GDP per Capita_9960.139039',
       'GDP per Capita_9965.635971',       'GDP per Capita_9966.047881',
       'GDP per Capita_9968.987254',       'GDP per Capita_9970.441715',
       'GDP per Capita_9984.146756',       'GDP per Capita_9997.686914']
Length: 3371, dtype: object

In [21]:
import numpy as np
#sort the list of features by the importance value in descending order
def double_sort(num_array, array_to_sort):
    for i in range(len(num_array)):
        swap = i + np.argmax(num_array[i:])
        (num_array[i], num_array[swap]) = (num_array[swap], num_array[i])
        (array_to_sort[i], array_to_sort[swap]) = (array_to_sort[swap], array_to_sort[i])
    return num_array, array_to_sort

In [22]:
feature_importance_sort, feature_sort = double_sort (feature_importance,  feature_list)
feature_sort

<PandasArray>
[                 'Life Expectancy', 'Goverment Expenditure per Capita',
   'Private Expenditure per Capita',              'Alcohol Consumption',
              'Tobacco Consumption',                        'sss_depth',
       'GDP per Capita_2167.202457',       'GDP per Capita_814.7729655',
       'GDP per Capita_939.3751427',       'GDP per Capita_1058.325559',
 ...
       'GDP per Capita_13825.35712',       'GDP per Capita_286.1078616',
       'GDP per Capita_2144.247803',        'GDP per Capita_29321.3254',
       'GDP per Capita_9965.635971',        'GDP per Capita_10249.9164',
        'GDP per Capita_3960.97533',       'GDP per Capita_9970.441715',
       'GDP per Capita_42174.66506',       'GDP per Capita_4218.444016']
Length: 3371, dtype: object